## Section 03: Flop & Turn Equity Calculation
**Abstract**: In this notebook, we calculate the flop equity for each player's hand based on various engineered features derived from the hole cards and flop cards. This involves grouping hands by these features and estimating the equity for each group, similar to the approach used for pre-flop equity calculations.

In [1]:
# Imports
import pandas as pd
from deuces import Evaluator

**Flop Equity Features**: We estimate flop equity by grouping hands based on other features that can be engineered from the hole cards and flop cards. These features include:
- Hand Classification: Pair, High Card, etc.
- Pocket Pair: Boolean indicating if the hole cards are a pocket pair.
- Suited: Boolean indicating if the hole cards are suited.
- Flush Potential: Complete, Open, Backdoor, or None.
- Straight Potential: Complete, Open, Gut-shot, Backdoor, or None.
- Overcards: Number of hole cards that are higher than the highest card on the flop.
- Board Texture: Number of unique suits/ranks on the board.
- Board Connectivity: Number of cards required to complete a straight on the board.

Functions to compute these features are defined in the `src.helpers` module.

In [2]:
"""
load long-form dataframe and remove all unnecessary columns, we are only interested in the player's hole cards, flop cards, flop evaluation, and showdown order
"""
hands_long = pd.read_pickle('../data/hands_long.pkl')
flop_hands = hands_long[[
    'hole_', 'flop', 'flop_eval_', 'showdown_order_', 'hand_class_'
]].copy()
# preview data
flop_hands.head()

,hole_,flop,flop_eval_,showdown_order_
0,"[533255, 67144223]","[16795671, 33573149, 16783383]",4310,2
1,"[4212241, 164099]","[268454953, 1082379, 134253349]",6322,4
2,"[268454953, 8394515]","[67119647, 81922, 1065995]",6428,5
3,"[8406803, 8398611]","[268446761, 4204049, 4212241]",3018,4
4,"[16787479, 279045]","[268454953, 4212241, 73730]",6588,8


In [3]:
"""
Add hand features for grouping by
"""

from src.hand_features import is_pocket_pair, is_suited, flush_potential, straight_potential, overcards, suit_texture, rank_texture, board_connectivity

flop_hands['pocket_pair'] = flop_hands['hole_'].apply(is_pocket_pair)
flop_hands['suited'] = flop_hands['hole_'].apply(is_suited)
flop_hands['flush_potential'] = flop_hands[['hole_', 'flop']].apply(
    lambda row: flush_potential(row['hole_'], row['flop']),
    axis=1
)
flop_hands['straight_potential'] = flop_hands[['hole_','flop']].apply(
    lambda row: straight_potential(row['hole_'], row['flop']),
    axis=1
)
flop_hands['overcards'] = flop_hands[['hole_','flop']].apply(
    lambda row: overcards(row['hole_'], row['flop']),
    axis=1
)
flop_hands['suit_texture'] = flop_hands['flop'].apply(suit_texture)
flop_hands['rank_texture'] = flop_hands['flop'].apply(rank_texture)
flop_hands['board_connectivity'] = flop_hands['flop'].apply(board_connectivity)
# drop unnecessary columns
flop_hands = flop_hands.drop(columns=['hole_', 'flop', 'flop_eval_'])
# preview data
flop_hands.head()

,showdown_order_,hand_class,pocket_pair,suited,flush_potential,straight_potential,overcards,suit_texture,rank_texture,board_connectivity
0,2,8,False,False,None,Backdoor,1,2,2,Low
1,4,9,False,False,Backdoor,None,0,2,3,Low
2,5,9,False,False,Backdoor,None,1,2,3,Low
3,4,7,True,False,Backdoor,None,0,2,2,Low
4,8,9,False,False,Backdoor,Backdoor,0,2,3,Low


Now we need to account for different player counts, from 2 to 9 players. We can do this differently from the pre-flop equity calculation, since we don't need to merge on any features. We can simply replicate the dataframe for each player count and assign the player count to a new column.

In [4]:
"""
multiply by number of players to account for different player counts
"""
flop_hands = pd.concat([flop_hands.assign(players=players) for players in range(2,10)], ignore_index=False)
# we should have 900k * 8 = 7.2 million rows now
flop_hands.info()

<class 'pandas.core.frame.DataFrame'>
Index: 7200000 entries, 0 to 899999
Data columns (total 11 columns):
 #   Column              Dtype 
---  ------              ----- 
 0   showdown_order_     int64 
 1   hand_class          int64 
 2   pocket_pair         bool  
 3   suited              bool  
 4   flush_potential     object
 5   straight_potential  object
 6   overcards           int64 
 7   suit_texture        int64 
 8   rank_texture        int64 
 9   board_connectivity  object
 10  players             int64 
dtypes: bool(2), int64(6), object(3)
memory usage: 563.0+ MB


Just like in the pre-flop equity calculation, we need to compute the total number of hands to consider for each player count, as well as the number of times a hand would win at showdown given its showdown order and player count. We can use the same combinatorial methods defined in the `src.helpers` module.

In [5]:
"""
compute equity features (total hands and wins)
"""

from src.equity_features import total_hands, wins

flop_hands['total_hands'] = flop_hands['players'].apply(total_hands)

flop_hands['wins'] = flop_hands[['showdown_order_','players']].apply(
    lambda row: wins(row['showdown_order_'], row['players']),
    axis=1
)
# preview data
flop_hands.head()

,showdown_order_,hand_class,pocket_pair,suited,flush_potential,straight_potential,overcards,suit_texture,rank_texture,board_connectivity,players,total_hands,wins
0,2,8,False,False,None,Backdoor,1,2,2,Low,2,8,7
1,4,9,False,False,Backdoor,None,0,2,3,Low,2,8,5
2,5,9,False,False,Backdoor,None,1,2,3,Low,2,8,4
3,4,7,True,False,Backdoor,None,0,2,2,Low,2,8,5
4,8,9,False,False,Backdoor,Backdoor,0,2,3,Low,2,8,1


In [6]:
"""
aggregate to get total wins and total hands for each condition
"""
flop_equity = flop_hands.groupby(
    ['players', 'hand_class', 'pocket_pair', 'suited', 'flush_potential',
     'straight_potential', 'overcards', 'suit_texture', 'rank_texture', 'board_connectivity']
).agg(
    total_wins=('wins', 'sum'),
    total_hands=('total_hands', 'sum'),
).reset_index()
# preview data
flop_equity.head()

,players,hand_class,pocket_pair,suited,flush_potential,straight_potential,overcards,suit_texture,rank_texture,board_connectivity,total_wins,total_hands
0,2,1,False,True,Complete,Complete,0,1,3,Low,40,40
1,2,1,False,True,Complete,Complete,1,1,3,Low,16,16
2,2,1,False,True,Complete,Complete,1,1,3,Medium,8,8
3,2,1,False,True,Complete,Complete,2,1,3,Medium,24,24
4,2,2,False,False,None,None,0,3,1,Low,336,336


In [7]:
# save locally
flop_equity.to_pickle('../data/flop_equity.pkl')
flop_equity.to_csv('../data/flop_equity.csv', index=False)

**Conclusion**: In this notebook, we successfully calculated the flop equity for various hand conditions by engineering relevant features from the hole and flop cards. We grouped the hands based on these features and estimated the equity for each group, similar to our approach for pre-flop equity calculations. The resulting flop equity data can be used to inform strategic decisions in poker gameplay.